In [3]:
!git config --global user.email "juanimendez100@gmail.com"
!git config --global user.name "Juan Méndez"
!git clone https://github.com/JuanMendez11/mendez_juan_tp_dt.git

Cloning into 'mendez_juan_tp_dt'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 182 (delta 64), reused 148 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (182/182), 11.06 MiB | 6.26 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [4]:
%cd mendez_juan_tp_dt/

/content/mendez_juan_tp_dt


In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 89.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 96.2 MB/s eta 0:00:00


In [6]:
!pip install -e .

Obtaining file:///content/mendez_juan_tp_dt
  Preparing metadata (setup.py) ... done
  Running setup.py develop for my_project


# Entrenamiento del modelo

In [7]:
from src.models.decision_transformer import DecisionTransformer

model = DecisionTransformer(
    num_items=752,
    num_groups=8,
    hidden_dim=128,
    n_layers=3,
    n_heads=4
)

print(f"Parámetros totales: {sum(p.numel() for p in model.parameters())}")
# Debería ser ~10-20M parámetros

Parámetros totales: 831728
